### Guardador de dados
Série de scripts para processar as imagens em *jpg* que saem do *coletor_de_dados.py* e armazená-las, assim como os labels, em arquivos binários, para posteriormente treinar a máquina. \
Obs.: Cada conjunto de imagens que serão processadas a cada iteração terão o mesmo label, definido na variável *data_labels*

In [ ]:
# Definição de variáveis

# MUDAR ESTAS
data_labels = 2        # Label das imagens

data_path = "../data/data.npy"
labels_path = "../data/labels.npy"
new_data_path = "../data/new_data.npy"


In [ ]:
# Carrega as fotos já existentes

import numpy as np

try: 
    old_data = np.load(data_path)
    old_labels = np.load(labels_path)
    print("Files loaded sucessfuly.")
except FileNotFoundError:
    print("Files not found. Starting from scratch...")
    old_data = np.array([])
    old_labels = np.array([])


print("Data loaded: ", old_data.shape)


In [ ]:
# Mostra quantos dados tem
somatorio = dict()
for dado, label in zip(old_data, old_labels):
    if label in somatorio:
        somatorio[label] += 1
    else:
        somatorio[label] = 1

print("Dados atuais:")
for key in somatorio:
    print(key, ': ', somatorio[key], sep='')

In [ ]:
# Carrega as fotos novas do arquivo
import cv2
import matplotlib.pyplot as plt
from utility import modify_images

# Carrega os dados
try:
    new_data = np.load(new_data_path)
    print("Successfully loaded new images.")
    print(new_data.shape)
except FileNotFoundError:
    print("File not found. No new data to append")


# Modifica cada uma das imagens criando uma função vetorizada
vect_modify_images = np.vectorize(modify_images, signature="(400, 400, 3)->(50, 50)")
new_data = vect_modify_images(new_data)

# mostra algumas imagens
fig, ax = plt.subplots(1, min(5, len(new_data)))
if len(new_data) > 1:
    for i, axes in enumerate(ax):
        # Mostra a imagem como matriz transposta (pois pygame pega assim)
        axes.imshow(new_data[i].transpose((1, 0)), cmap='gray')
        axes.set_title(f"Img {i}")
else:
    # Mostra a imagem como matriz transposta (pois pygame pega assim)
    ax.imshow(new_data[0].transpose((1, 0)), cmap='gray')
    ax.set_title("Img 0")

print(new_data.shape)

# Cria o vetor de novos labels
new_labels = np.full(len(new_data), data_labels)


In [ ]:
# Junta os dados

if len(old_data): # junta apenas se tiver dados antigos
    new_data = np.append(old_data, new_data, axis=0)
    new_labels = np.append(old_labels, new_labels, axis=0)


print("data shape:", new_data.shape)
print("labels shape:", new_labels.shape)

In [ ]:
# Finalização
import os
# Coloca os novos dados no arquivo
np.save(data_path, new_data)
np.save(labels_path, new_labels)

# E apaga os dados já salvos
os.remove(new_data_path)